# EDA BATTEL POKEMON

## Imports

**Version of python = 3.13.2**

**Versions of package**
- *Pandas*     = 3.0.0
- *Numpy*      = 2.4.2
- *Matplotlib* = 3.10.8

In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

pd.set_option("display.max_columns", 50)
pd.set_option("display.width", 120)

# Render plots inline
%matplotlib inline

In [12]:
df= pd.read_csv("../data/pokemon.csv")
df.head()

,#,Name,Type 1,Type 2,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,80,82,83,100,100,80,1,False
3,4,Mega Venusaur,Grass,Poison,80,100,123,122,120,80,1,False
4,5,Charmander,Fire,NaN,39,52,43,60,50,65,1,False


In [29]:
# Valors nulls in the dataset 
nls = df.isnull().sum()
mask = nls > 0

# Mean values of the columns
means = df.describe().mean().drop(labels=["#", "Generation"])

# Prints information about the dataset
print("Dataset shape: " + str(df.shape))

print("\nNull values per column: \n\n" + str(nls[mask]) +"\n")

print("Mean values per column: \n")

print(means)


Dataset shape: (800, 12)

Null values per column: 

Type 2    386
dtype: int64

Mean values per column: 

HP         168.224177
Attack     167.057327
Defense    168.753250
Sp. Atk    164.911537
Sp. Def    169.966427
Speed      160.292247
dtype: float64
